In [3]:
import sys
import os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount("/content/drive")
    %cd /content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/notebooks

    dname = "nb009-cv"
    os.makedirs(dname, exist_ok=True)


    !pip install --upgrade --force-reinstall --no-deps kaggle
    import json
    f = open("/content/drive/MyDrive/colab_notebooks/kaggle/kaggle.json", "r")
    json_data = json.load(f)
    os.environ["KAGGLE_USERNAME"] = json_data["username"]
    os.environ["KAGGLE_KEY"] = json_data["key"]


    import yaml
    from pprint import pprint
    with open('../config/config.yml') as file:
        CFG = yaml.load(file)
    pprint(CFG)

Mounted at /content/drive
/content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/notebooks
     |████████████████████████████████| 61kB 5.7MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=186503576026e47af93375a50f05d4416d68248c88b60ea25ea0387f4fe91e21
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
{'cv_debug': False,
 'debug': True,
 'embedding_dim': 100,
 'epochs': 2,
 'is_single': False,
 'learning_rate': 2e-05,
 'max_len': 128,
 'tags_vals': 'o o-dataset pad',
 'test_batch_size': 64,
 'text_len': 0,
 'train': True,
 'train_batch_size': 32,
 'unique_key': 'pub_title',
 'use_cosine': True,
 'use_pos': False,
 'valid_batch_size': 64}


In [4]:
import numpy as np
import pandas as pd

from joblib import Parallel, delayed
#from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm
import spacy
from collections import defaultdict
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
import pickle
import gc
import random
from pprint import pprint

In [5]:
if 'google.colab' in sys.modules:
    use_cosine = CFG["use_cosine"]
    is_single = CFG["is_single"]
    debug = CFG["cv_debug"]
    unique_key = CFG["unique_key"]
elif 'kaggle_web_client' in sys.modules:
    use_cosine = False
    is_single = False
    debug = False
    unique_key = "pub_title"

In [6]:
df = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/train.csv")
labels = np.sort(df["cleaned_label"].unique())
labels = list(labels)

if debug:
    #labels = labels[:20]
    labels = random.sample(labels, 20)
    print("Debug with the length of", len(labels))
    pprint(labels)

In [7]:
len(df)

19661

In [8]:
num_id = df[unique_key].nunique()
print("num_id: ", num_id)

num_id:  14271


In [9]:
# 一つのIdで2つ以上のcleaned_labelを持つもの:
dup_ids = df[df.duplicated(subset=unique_key)][unique_key].unique()
tmp = df[df[unique_key].isin(dup_ids)].sort_values(by=unique_key)
tmp

,Id,pub_title,dataset_title,dataset_label,cleaned_label
15292,eebbbca1-d530-4d4a-b1c5-a1f157fd931c,15 Developing new employment and compensation...,Survey of Earned Doctorates,Survey of Earned Doctorates,survey of earned doctorates
15293,eebbbca1-d530-4d4a-b1c5-a1f157fd931c,15 Developing new employment and compensation...,Survey of Doctorate Recipients,Survey of Doctorate Recipients,survey of doctorate recipients
15294,eebbbca1-d530-4d4a-b1c5-a1f157fd931c,15 Developing new employment and compensation...,Survey of Graduate Students and Postdoctorates...,Survey of Graduate Students and Postdoctorates...,survey of graduate students and postdoctorates...
15339,66236b50-748c-49d3-9bb7-5d86d4a4f9d1,2007 R&D Satellite Account Methodologies: Cur...,Survey of Industrial Research and Development,NSF Survey of Industrial Research and Development,nsf survey of industrial research and development
15338,66236b50-748c-49d3-9bb7-5d86d4a4f9d1,2007 R&D Satellite Account Methodologies: Cur...,Survey of Industrial Research and Development,Survey of Industrial Research and Development,survey of industrial research and development
...,...,...,...,...,...
10483,b4e3602c-8396-4143-bb2e-a8bfb45575ff,β-Amyloid is Associated with Aberrant Metaboli...,Alzheimer's Disease Neuroimaging Initiative (A...,Alzheimer's Disease Neuroimaging Initiative (A...,alzheimer s disease neuroimaging initiative adni
6056,13c5b4e7-d711-4600-8034-b2071cf7a90e,“Big Data” in neuroscience: open door to a mor...,Alzheimer's Disease Neuroimaging Initiative (A...,ADNI,adni
9081,13c5b4e7-d711-4600-8034-b2071cf7a90e,“Big Data” in neuroscience: open door to a mor...,Alzheimer's Disease Neuroimaging Initiative (A...,Alzheimer's Disease Neuroimaging Initiative (A...,alzheimer s disease neuroimaging initiative adni
6578,7d03cdff-c4dc-49a4-964f-e7e4504cecea,“Radiotranscriptomics”: A synergy of imaging a...,Alzheimer's Disease Neuroimaging Initiative (A...,ADNI,adni


***
# Cosine Similarity Operation

In [10]:
# labels2manualcat
labels2manualcat = {
    '2019 ncov complete genome sequences':'covid 19 genome sequences',
    '2019 ncov genome sequence':'covid 19 genome sequences',
    '2019 ncov genome sequences':'covid 19 genome sequences',
    'adni':'alzheimers disease neuroimaging initiative',
    'advanced national seismic system anss comprehensive catalog comcat ':'seismic system comprehensive catalog',
    'advanced national seismic system comprehensive catalog':'seismic system comprehensive catalog',
    'aging integrated database':'aging integrated database',
    'aging integrated database agid ':'aging integrated database',
    'agricultural resource management survey':'agricultural resources management survey',
    'agricultural resources management survey':'agricultural resources management survey',
    'alzheimer s disease neuroimaging initiative adni ':'alzheimers disease neuroimaging initiative',
    'alzheimers disease neuroimaging initiative':'alzheimers disease neuroimaging initiative',
    'anss comprehensive catalog':'seismic system comprehensive catalog',
    'anss comprehensive earthquake catalog':'seismic system comprehensive catalog',
    'arms farm financial and crop production practices':'arms farm financial and crop production practices',
    'baccalaureate and beyond':'baccalaureate and beyond longitudinal study',
    'baccalaureate and beyond longitudinal study':'baccalaureate and beyond longitudinal study',
    'baltimore longitudinal study of aging':'baltimore longitudinal study of aging',
    'baltimore longitudinal study of aging blsa ':'baltimore longitudinal study of aging',
    'beginning postsecondary student':'beginning postsecondary students',
    'beginning postsecondary students':'beginning postsecondary students',
    'beginning postsecondary students longitudinal study':'beginning postsecondary students',
    'cas covid 19 antiviral candidate compounds data':'cas covid 19 antiviral candidate compounds dataset',
    'cas covid 19 antiviral candidate compounds data set':'cas covid 19 antiviral candidate compounds dataset',
    'cas covid 19 antiviral candidate compounds dataset':'cas covid 19 antiviral candidate compounds dataset',
    'cccsl':'complexity science hub covid 19 control strategies list',
    'census of agriculture':'census of agriculture',
    'characterizing health associated risks and your baseline disease in sars cov 2':'characterizing health associated risks and your baseline disease in sars cov 2',
    'characterizing health associated risks and your baseline disease in sars cov 2 charybdis ':'characterizing health associated risks and your baseline disease in sars cov 2',
    'coastal change analysis program':'coastal change analysis program',
    'coastal change analysis program land cover':'coastal change analysis program',
    'common core of data':'common core of data',
    'complexity science hub covid 19 control strategies list':'complexity science hub covid 19 control strategies list',
    'complexity science hub covid 19 control strategies list cccsl ':'complexity science hub covid 19 control strategies list',
    'covid 19 death data':'covid 19 death data',
    'covid 19 genome sequence':'covid 19 genome sequences',
    'covid 19 genome sequences':'covid 19 genome sequences',
    'covid 19 image data collection':'covid 19 image data collection',
    'covid 19 open research data':'covid 19 open research dataset',
    'covid 19 open research dataset':'covid 19 open research dataset',
    'covid 19 open research dataset cord 19 ':'covid 19 open research dataset',
    'covid open research dataset':'covid 19 open research dataset',
    'early childhood longitudinal study':'early childhood longitudinal study',
    'education longitudinal study':'education longitudinal study',
    'ffrdc research and development survey':'ffrdc research and development survey',
    'genome sequence of 2019 ncov':'covid 19 genome sequences',
    'genome sequence of covid 19':'covid 19 genome sequences',
    'genome sequence of sars cov 2':'covid 19 genome sequences',
    'genome sequences of 2019 ncov':'covid 19 genome sequences',
    'genome sequences of covid 19':'covid 19 genome sequences',
    'genome sequences of sars cov 2':'covid 19 genome sequences',
    'high school longitudinal study':'high school longitudinal study',
    'higher education research and development survey':'higher education research and development survey',
    'ibtracs':'international best track archive for climate stewardship',
    'international best track archive for climate stewardship':'international best track archive for climate stewardship',
    'jh crown registry':'jh crown registry',
    'nass census of agriculture':'census of agriculture',
    'national assessment of education progress':'national assessment of education progress',
    'national center for education statistics common core of data':'common core of data',
    'national center for science and engineering statistics higher education research and development survey':'higher education research and development survey',
    'national center for science and engineering statistics survey of doctorate recipients':'survey of doctorate recipients',
    'national center for science and engineering statistics survey of earned doctorates':'survey of earned doctorates',
    'national center for science and engineering statistics survey of graduate students and postdoctorates in science and engineering':'survey of graduate students and postdoctorates in science and engineering',
    'national center for science and engineering statistics survey of industrial research and development':'survey of industrial research and development',
    'national center for science and engineering statistics survey of science and engineering research facilities':'survey of science and engineering research facilities',
    'national education longitudinal study':'education longitudinal study',
    'national oceanic and atmospheric administration c cap':'coastal change analysis program',
    'national oceanic and atmospheric administration optimum interpolation sea surface temperature':'optimum interpolation sea surface temperature',
    'national oceanic and atmospheric administration world ocean database':'world ocean database',
    'national science foundation higher education research and development survey':'higher education research and development survey',
    'national science foundation survey of doctorate recipients':'survey of doctorate recipients',
    'national science foundation survey of earned doctorates':'survey of earned doctorates',
    'national science foundation survey of graduate students and postdoctorates in science and engineering':'survey of graduate students and postdoctorates in science and engineering',
    'national science foundation survey of industrial research and development':'survey of industrial research and development',
    'national science foundation survey of science and engineering research facilities':'survey of science and engineering research facilities',
    'national teacher and principal survey':'teacher and principal survey',
    'national water level observation network':'water level observation network',
    'national weather service nws storm surge risk':'storm surge risk',
    'nces common core of data':'common core of data',
    'ncses survey of doctorate recipients':'survey of doctorate recipients',
    'ncses survey of graduate students and postdoctorates in science and engineering':'survey of graduate students and postdoctorates in science and engineering',
    'noaa c cap':'coastal change analysis program',
    'noaa international best track archive for climate stewardship':'international best track archive for climate stewardship',
    'noaa national water level observation network':'water level observation network',
    'noaa optimum interpolation sea surface temperature':'optimum interpolation sea surface temperature',
    'noaa sea lake and overland surges from hurricanes':'storm surge risk',
    'noaa storm surge inundation':'storm surge risk',
    'noaa tidal station':'tide station',
    'noaa tide gauge':'tide station',
    'noaa tide station':'tide station',
    'noaa water level station':'water level observation network',
    'noaa world ocean database':'world ocean database',
    'north american breeding bird survey':'breeding bird survey',
    'north american breeding bird survey bbs ':'breeding bird survey',
    'nsf ffrdc research and development survey':'ffrdc research and development survey',
    'nsf higher education research and development survey':'higher education research and development survey',
    'nsf survey of earned doctorates':'survey of earned doctorates',
    'nsf survey of graduate students and postdoctorates in science and engineering':'survey of graduate students and postdoctorates in science and engineering',
    'nsf survey of industrial research and development':'survey of industrial research and development',
    'nsf survey of science and engineering research facilities':'survey of science and engineering research facilities',
    'optimum interpolation sea surface temperature':'optimum interpolation sea surface temperature',
    'our world in data':'covid 19 our world in data',
    'our world in data covid 19':'covid 19 our world in data',
    'our world in data covid 19 dataset':'covid 19 our world in data',
    'program for the international assessment of adult competencies':'program for the international assessment of adult competencies',
    'rsna international covid 19 open radiology database':'rsna international covid 19 open radiology database',
    'rsna international covid 19 open radiology database ricord ':'rsna international covid 19 open radiology database',
    'rsna international covid open radiology database':'rsna international covid 19 open radiology database',
    'rural urban continuum codes':'rural urban continuum codes',
    'sars cov 2 complete genome sequence':'covid 19 genome sequences',
    'sars cov 2 complete genome sequences':'covid 19 genome sequences',
    'sars cov 2 full genome sequence':'covid 19 genome sequences',
    'sars cov 2 full genome sequences':'covid 19 genome sequences',
    'sars cov 2 genome sequence':'covid 19 genome sequences',
    'sars cov 2 genome sequences':'covid 19 genome sequences',
    'school survey on crime and safety':'school survey on crime and safety',
    'sea surface temperature optimum interpolation':'optimum interpolation sea surface temperature',
    'slosh model':'storm surge risk',
    'survey of doctorate recipients':'survey of doctorate recipients',
    'survey of earned doctorates':'survey of earned doctorates',
    'survey of graduate students and postdoctorates in science and engineering':'survey of graduate students and postdoctorates in science and engineering',
    'survey of industrial research and development':'survey of industrial research and development',
    'survey of science and engineering research facilities':'survey of science and engineering research facilities',
    'survey of state government research and development':'survey of state government research and development',
    'the national institute on aging genetics of alzheimer s disease data storage site':'the national institute on aging genetics of alzheimer s disease data storage site',
    'the national institute on aging genetics of alzheimer s disease data storage site niagads ':'the national institute on aging genetics of alzheimer s disease data storage site',
    'trends in international mathematics and science study':'trends in international mathematics and science study',
    'usda census of agriculture':'census of agriculture',
    'usgs north american breeding bird survey':'breeding bird survey',
    'world ocean database':'world ocean database'
}

print(len(set(labels2manualcat.values())))

46


In [11]:
df["cleaned_label_cmprsd"] = df["cleaned_label"].apply(lambda x: labels2manualcat.get(x))

In [12]:
len(df)

19661

In [13]:
# 同一Idで複数のcleaned_labelを持つが, cleaned_label_cmprsdでみると重複しているものについてはdrop
df = df.drop_duplicates(subset=[unique_key, "cleaned_label_cmprsd"]).reset_index(drop=True)

In [14]:
# 一つのIdで2つ以上のcleaned_label_cmprsdを持つもの:
dup_ids = df[df.duplicated(subset=[unique_key, "cleaned_label_cmprsd"])][unique_key].unique()
tmp = df[df[unique_key].isin(dup_ids)].sort_values(by=unique_key)
tmp

,Id,pub_title,dataset_title,dataset_label,cleaned_label,cleaned_label_cmprsd


In [15]:
len(df)

15005

In [16]:
df.drop("cleaned_label_cmprsd", axis=1, inplace=True)

In [17]:
def cosim(doc1, doc2):
    """
    Args:
        doc1: str
        doc2: str
    Returns:
        sim: float
    """
    nlp = spacy.load("en_core_web_sm")  # make sure to use larger package!
    # reference: https://github.com/explosion/spaCy/issues/3552

    # Rule based categorization
    doc1 = labels2manualcat.get(doc1)
    doc2 = labels2manualcat.get(doc2)
    
    doc1 = nlp(doc1)
    doc2 = nlp(doc2)
    
    sim = doc1.similarity(doc2)
    return sim

In [ ]:
%%time
if use_cosine:
    if is_single:
        outs = []
        with tqdm(total=len(labels)*len(labels)) as pbar:
            for doc1 in tqdm(labels):
                for doc2 in tqdm(labels):
                    outs.append(cosim(doc1, doc2))
                    pbar.update(1)
    else:
        outs = Parallel(n_jobs=-1)(delayed(cosim)(doc1, doc2) for doc2 in tqdm(labels) for doc1 in tqdm(labels))

    outs = np.array(outs).reshape((len(labels), len(labels)))
    np.fill_diagonal(outs, 0)
    
    if 'google.colab' in sys.modules:
        pickle.dump(outs, open(f"./{dname}/outs.pkl", "wb"))
    elif 'kaggle_web_client' in sys.modules:
        pickle.dump(outs, open(f"./outs.pkl", "wb"))

    del outs
    gc.collect()




CPU times: user 2min 12s, sys: 6.98 s, total: 2min 19s
Wall time: 1h 15min 6s


***
# (deprecated) Simple String Operation

In [18]:
if not use_cosine:
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    
    # Lemmatization
    # reference: https://stackoverflow.com/questions/38763007/how-to-use-spacy-lemmatizer-to-get-a-word-into-basic-form/48948642
    lab2lem = dict()
    for lab in labels:
        doc = nlp(lab)
        lem = " ".join([token.lemma_ for token in doc])
        lab2lem[lab] = lem
        
    lemmas = [lab2lem[label] for label in labels]
    
    
    outs = []
    for doc1 in lemmas:
        for doc2 in lemmas:
            outs.append(doc1 in doc2)
    
    outs = np.array(outs).reshape((len(lemmas), len(lemmas)))
    np.fill_diagonal(outs, False)

    if 'google.colab' in sys.modules:
        pickle.dump(outs, open(f"./{dname}/outs.pkl", "wb"))
    elif 'kaggle_web_client' in sys.modules:
        pickle.dump(outs, open(f"./outs.pkl", "wb"))

    del outs
    gc.collect()

***
# Graph

In [19]:
if 'google.colab' in sys.modules:
    outs = pickle.load(open(f"./{dname}/outs.pkl", "rb"))
elif 'kaggle_web_client' in sys.modules:
    outs = pickle.load(open(f"./outs.pkl", "rb"))

labels = np.array(labels)

if debug:
    print(np.around(outs, 2))

In [20]:
if use_cosine:
    thres = 0.95
    if debug:
        print("labels:\n", labels)
        print()
    for i,label in enumerate(labels):
        print(i)
        similars = labels[outs[i, :] >= thres]
        print(f"{label}: {similars}")
        if debug:
            print("similarities: ", np.around(outs[i,:], 3))
        print()

0
2019 ncov complete genome sequences: ['2019 ncov genome sequence' '2019 ncov genome sequences'
 'covid 19 genome sequence' 'covid 19 genome sequences'
 'genome sequence of 2019 ncov' 'genome sequence of covid 19'
 'genome sequence of sars cov 2' 'genome sequences of 2019 ncov'
 'genome sequences of covid 19' 'genome sequences of sars cov 2'
 'sars cov 2 complete genome sequence'
 'sars cov 2 complete genome sequences' 'sars cov 2 full genome sequence'
 'sars cov 2 full genome sequences' 'sars cov 2 genome sequence'
 'sars cov 2 genome sequences']

1
2019 ncov genome sequence: ['2019 ncov complete genome sequences' '2019 ncov genome sequences'
 'covid 19 genome sequence' 'covid 19 genome sequences'
 'genome sequence of 2019 ncov' 'genome sequence of covid 19'
 'genome sequence of sars cov 2' 'genome sequences of 2019 ncov'
 'genome sequences of covid 19' 'genome sequences of sars cov 2'
 'sars cov 2 complete genome sequence'
 'sars cov 2 complete genome sequences' 'sars cov 2 full gen

In [21]:
if use_cosine:
    outs = outs > thres

In [22]:
# function for adding edge to graph
graph = defaultdict(list)

def addEdge(graph,u,v):
    graph[u].append(v)

# definition of function
def generate_edges(graph):
    edges = []
    # for each node in graph
    for node in graph:
        # for each neighbour node of a single node
        for neighbour in graph[node]:
            # if edge exists then append
            edges.append((node, neighbour))
    return edges

In [23]:
#labels = np.array(labels)

for i in range(len(labels)):
    u = labels[i]
    vs = labels[outs[i, :]]
#     print("doc:\n", u)
#     print("Similar docs:\n", vs)
#     print()
    for v in vs:
        addEdge(graph, u, v)
        
edges = generate_edges(graph)

G = nx.Graph()
G.add_edges_from(edges)

In [ ]:
#nx.draw(G, with_labels=True)

In [ ]:
# for cc in nx.connected_components(G):
#     print(cc)

In [24]:
mycat = dict()
for label in labels:
    try:
        ncc = list(nx.node_connected_component(G, label)) # Set[str] -> List[str]
        ncc = list(set([labels2manualcat.get(label) for label in ncc]))
        ncc.sort()
        #print(type(ncc))
        #print(f"label={label}: ncc={ncc}")
        print(ncc)
        cat = " + ".join(ncc)
        #print(f"label: {label},\n cat: {cat}\n")
    except KeyError:
        cat = labels2manualcat.get(label)
    mycat[label] = cat

['covid 19 genome sequences']
['covid 19 genome sequences']
['covid 19 genome sequences']
['alzheimers disease neuroimaging initiative']
['seismic system comprehensive catalog']
['seismic system comprehensive catalog']
['aging integrated database']
['aging integrated database']
['agricultural resources management survey']
['agricultural resources management survey']
['alzheimers disease neuroimaging initiative']
['alzheimers disease neuroimaging initiative']
['seismic system comprehensive catalog']
['seismic system comprehensive catalog']
['baccalaureate and beyond longitudinal study']
['baccalaureate and beyond longitudinal study']
['baltimore longitudinal study of aging']
['baltimore longitudinal study of aging']
['beginning postsecondary students']
['beginning postsecondary students']
['beginning postsecondary students']
['cas covid 19 antiviral candidate compounds dataset']
['cas covid 19 antiviral candidate compounds dataset']
['cas covid 19 antiviral candidate compounds dataset']

In [25]:
mycat

{'2019 ncov complete genome sequences': 'covid 19 genome sequences',
 '2019 ncov genome sequence': 'covid 19 genome sequences',
 '2019 ncov genome sequences': 'covid 19 genome sequences',
 'adni': 'alzheimers disease neuroimaging initiative',
 'advanced national seismic system anss comprehensive catalog comcat ': 'seismic system comprehensive catalog',
 'advanced national seismic system comprehensive catalog': 'seismic system comprehensive catalog',
 'aging integrated database': 'aging integrated database',
 'aging integrated database agid ': 'aging integrated database',
 'agricultural resource management survey': 'agricultural resources management survey',
 'agricultural resources management survey': 'agricultural resources management survey',
 'alzheimer s disease neuroimaging initiative adni ': 'alzheimers disease neuroimaging initiative',
 'alzheimers disease neuroimaging initiative': 'alzheimers disease neuroimaging initiative',
 'anss comprehensive catalog': 'seismic system compr

In [26]:
print("number of mycat: ", len(set(mycat.values())))
set(mycat.values())

number of mycat:  46


{'aging integrated database',
 'agricultural resources management survey',
 'alzheimers disease neuroimaging initiative',
 'arms farm financial and crop production practices',
 'baccalaureate and beyond longitudinal study',
 'baltimore longitudinal study of aging',
 'beginning postsecondary students',
 'breeding bird survey',
 'cas covid 19 antiviral candidate compounds dataset',
 'census of agriculture',
 'characterizing health associated risks and your baseline disease in sars cov 2',
 'coastal change analysis program',
 'common core of data',
 'complexity science hub covid 19 control strategies list',
 'covid 19 death data',
 'covid 19 genome sequences',
 'covid 19 image data collection',
 'covid 19 open research dataset',
 'covid 19 our world in data',
 'early childhood longitudinal study',
 'education longitudinal study',
 'ffrdc research and development survey',
 'high school longitudinal study',
 'higher education research and development survey',
 'international best track arch

***
# CV

In [27]:
df["pub_category"] = df["cleaned_label"].apply(lambda x: mycat.get(x))
df["pub_category"].fillna("other", inplace=True)

Aggregate multiple pub_categories if exists.

In [28]:
# 一つのIdで2つ以上のcleaned_labelを持つもの:
dup_ids = df[df.duplicated(subset=unique_key)][unique_key].unique()
tmp = df[df[unique_key].isin(dup_ids)].sort_values(by=unique_key)
tmp

,Id,pub_title,dataset_title,dataset_label,cleaned_label,pub_category
11290,eebbbca1-d530-4d4a-b1c5-a1f157fd931c,15 Developing new employment and compensation...,Survey of Graduate Students and Postdoctorates...,Survey of Graduate Students and Postdoctorates...,survey of graduate students and postdoctorates...,survey of graduate students and postdoctorates...
11289,eebbbca1-d530-4d4a-b1c5-a1f157fd931c,15 Developing new employment and compensation...,Survey of Doctorate Recipients,Survey of Doctorate Recipients,survey of doctorate recipients,survey of doctorate recipients
11288,eebbbca1-d530-4d4a-b1c5-a1f157fd931c,15 Developing new employment and compensation...,Survey of Earned Doctorates,Survey of Earned Doctorates,survey of earned doctorates,survey of earned doctorates
10815,6208c990-cd08-4e9b-8f35-3cb013bb342a,A Comparison of Federally Funded Sample Surve...,High School Longitudinal Study,High School Longitudinal Study,high school longitudinal study,high school longitudinal study
10814,6208c990-cd08-4e9b-8f35-3cb013bb342a,A Comparison of Federally Funded Sample Surve...,Beginning Postsecondary Student,Beginning Postsecondary Students,beginning postsecondary students,beginning postsecondary students
...,...,...,...,...,...,...
10417,f93717d9-2206-4e3d-b63b-f890cced92c0,Environmental Impacts of Sea Level Rise in th...,"Sea, Lake, and Overland Surges from Hurricanes",SLOSH model,slosh model,storm surge risk
12122,6b0d6131-2ffe-4652-88be-74bb4bb6f424,Farm profits and adoption of precision agricu...,Agricultural Resource Management Survey,Agricultural Resource Management Survey,agricultural resource management survey,agricultural resources management survey
12123,6b0d6131-2ffe-4652-88be-74bb4bb6f424,Farm profits and adoption of precision agricu...,Agricultural Resource Management Survey,ARMS Farm Financial and Crop Production Practices,arms farm financial and crop production practices,arms farm financial and crop production practices
12139,dbfb1ae1-38aa-4b86-bc54-e2dbb4611cab,Structure and finances of US farms: family fa...,Agricultural Resource Management Survey,ARMS Farm Financial and Crop Production Practices,arms farm financial and crop production practices,arms farm financial and crop production practices


In [29]:
df[df["Id"] == "00f97f4b-39e6-4b02-ab74-c1d366d3e371"]

,Id,pub_title,dataset_title,dataset_label,cleaned_label,pub_category
10230,00f97f4b-39e6-4b02-ab74-c1d366d3e371,Storm Surge Hazard Database for the United St...,"Sea, Lake, and Overland Surges from Hurricanes",SLOSH model,slosh model,storm surge risk
10231,00f97f4b-39e6-4b02-ab74-c1d366d3e371,Storm Surge Hazard Database for the United St...,NOAA Tide Gauge,NOAA tide station,noaa tide station,tide station


In [30]:
tmp = df.groupby(unique_key).apply(lambda r: "|".join(list(set(r["cleaned_label"])))).to_frame(name="cleaned_label").reset_index(drop=False)
_tm = df.groupby(unique_key).apply(lambda r: list(set(r["dataset_label"]))).to_frame(name="dataset_label").reset_index(drop=False)
#tmp = tmp.join(_tm, on=unique_key, how='left')
tmp = pd.merge(tmp, _tm, on=unique_key, how='left')
tmp


,pub_title,cleaned_label,dataset_label
0,15 Developing new employment and compensation...,survey of doctorate recipients|survey of gradu...,"[Survey of Doctorate Recipients, Survey of Ear..."
1,20 Careers of Korean Ph Ds with Degrees of Fo...,survey of earned doctorates,[Survey of Earned Doctorates]
2,2003 National Survey of College Graduates Non...,survey of doctorate recipients,[Survey of Doctorate Recipients]
3,2006 farmer investment survey,agricultural resource management survey,[Agricultural Resource Management Survey]
4,2007 R&D Satellite Account Methodologies: Cur...,survey of industrial research and development,[Survey of Industrial Research and Development]
...,...,...,...
14266,“Tomorrow Never Dies”: Recent Advances in Diag...,genome sequence of sars cov 2,[genome sequence of SARS-CoV-2]
14267,“True to Myself”: Transforming Secondary Mathe...,trends in international mathematics and scienc...,[Trends in International Mathematics and Scien...
14268,"“Update VISION ON COVID-19: Structure, immune ...",genome sequence of covid 19,[genome sequence of COVID-19]
14269,“Which Child Left Behind”: Historical Issues R...,trends in international mathematics and scienc...,[Trends in International Mathematics and Scien...


In [31]:
_tm = df.groupby(unique_key).apply(lambda r: list(set(r["dataset_title"]))).to_frame(name="dataset_title").reset_index(drop=False)
#tmp = tmp.join(_tm, on=unique_key, how='left')
tmp = pd.merge(tmp, _tm, on=unique_key, how='left')

if unique_key != "Id":
    _tm = df.groupby(unique_key).apply(lambda r: list(set(r["Id"]))[0]).to_frame(name="Id").reset_index(drop=False)
else:
    _tm = df.groupby(unique_key).apply(lambda r: list(set(r["pub_title"]))[0]).to_frame(name="pub_title").reset_index(drop=False)
#tmp = tmp.join(_tm, on=unique_key, how='left')
tmp = pd.merge(tmp, _tm, on=unique_key, how='left')

tmp

,pub_title,cleaned_label,dataset_label,dataset_title,Id
0,15 Developing new employment and compensation...,survey of doctorate recipients|survey of gradu...,"[Survey of Doctorate Recipients, Survey of Ear...","[Survey of Doctorate Recipients, Survey of Ear...",eebbbca1-d530-4d4a-b1c5-a1f157fd931c
1,20 Careers of Korean Ph Ds with Degrees of Fo...,survey of earned doctorates,[Survey of Earned Doctorates],[Survey of Earned Doctorates],6bfda128-a0c0-41ba-96a8-ec62731124d9
2,2003 National Survey of College Graduates Non...,survey of doctorate recipients,[Survey of Doctorate Recipients],[Survey of Doctorate Recipients],780ded05-0a00-4249-8eb6-7bde14db4f76
3,2006 farmer investment survey,agricultural resource management survey,[Agricultural Resource Management Survey],[Agricultural Resource Management Survey],cddb661e-0e54-49a3-801c-76687eb25333
4,2007 R&D Satellite Account Methodologies: Cur...,survey of industrial research and development,[Survey of Industrial Research and Development],[Survey of Industrial Research and Development],66236b50-748c-49d3-9bb7-5d86d4a4f9d1
...,...,...,...,...,...
14266,“Tomorrow Never Dies”: Recent Advances in Diag...,genome sequence of sars cov 2,[genome sequence of SARS-CoV-2],[SARS-CoV-2 genome sequence],1113ef38-b394-4519-aa89-0c4f34da6f49
14267,“True to Myself”: Transforming Secondary Mathe...,trends in international mathematics and scienc...,[Trends in International Mathematics and Scien...,[Trends in International Mathematics and Scien...,6ceabf84-a4ec-46d7-8300-87a619382d4c
14268,"“Update VISION ON COVID-19: Structure, immune ...",genome sequence of covid 19,[genome sequence of COVID-19],[SARS-CoV-2 genome sequence],6fc05d28-d8df-4fbc-a1eb-43f1a9b95720
14269,“Which Child Left Behind”: Historical Issues R...,trends in international mathematics and scienc...,[Trends in International Mathematics and Scien...,[Trends in International Mathematics and Scien...,fa5280d4-5a11-4d06-b2aa-bc7a3bf0edd9


In [ ]:
# tmp = df[["Id", "cleaned_label"]].drop_duplicates(subset="Id", keep='first')
# tmp

In [32]:
s = df.groupby(unique_key).apply(lambda r: " + ".join(list(set(r["pub_category"]))))
s.name = "pub_category"
tmp = tmp.join(s, on=unique_key, how='left')
tmp

,pub_title,cleaned_label,dataset_label,dataset_title,Id,pub_category
0,15 Developing new employment and compensation...,survey of doctorate recipients|survey of gradu...,"[Survey of Doctorate Recipients, Survey of Ear...","[Survey of Doctorate Recipients, Survey of Ear...",eebbbca1-d530-4d4a-b1c5-a1f157fd931c,survey of doctorate recipients + survey of gra...
1,20 Careers of Korean Ph Ds with Degrees of Fo...,survey of earned doctorates,[Survey of Earned Doctorates],[Survey of Earned Doctorates],6bfda128-a0c0-41ba-96a8-ec62731124d9,survey of earned doctorates
2,2003 National Survey of College Graduates Non...,survey of doctorate recipients,[Survey of Doctorate Recipients],[Survey of Doctorate Recipients],780ded05-0a00-4249-8eb6-7bde14db4f76,survey of doctorate recipients
3,2006 farmer investment survey,agricultural resource management survey,[Agricultural Resource Management Survey],[Agricultural Resource Management Survey],cddb661e-0e54-49a3-801c-76687eb25333,agricultural resources management survey
4,2007 R&D Satellite Account Methodologies: Cur...,survey of industrial research and development,[Survey of Industrial Research and Development],[Survey of Industrial Research and Development],66236b50-748c-49d3-9bb7-5d86d4a4f9d1,survey of industrial research and development
...,...,...,...,...,...,...
14266,“Tomorrow Never Dies”: Recent Advances in Diag...,genome sequence of sars cov 2,[genome sequence of SARS-CoV-2],[SARS-CoV-2 genome sequence],1113ef38-b394-4519-aa89-0c4f34da6f49,covid 19 genome sequences
14267,“True to Myself”: Transforming Secondary Mathe...,trends in international mathematics and scienc...,[Trends in International Mathematics and Scien...,[Trends in International Mathematics and Scien...,6ceabf84-a4ec-46d7-8300-87a619382d4c,trends in international mathematics and scienc...
14268,"“Update VISION ON COVID-19: Structure, immune ...",genome sequence of covid 19,[genome sequence of COVID-19],[SARS-CoV-2 genome sequence],6fc05d28-d8df-4fbc-a1eb-43f1a9b95720,covid 19 genome sequences
14269,“Which Child Left Behind”: Historical Issues R...,trends in international mathematics and scienc...,[Trends in International Mathematics and Scien...,[Trends in International Mathematics and Scien...,fa5280d4-5a11-4d06-b2aa-bc7a3bf0edd9,trends in international mathematics and scienc...


In [43]:
def graph_pub_cat(cat1, cat2, graph):
    """
    Args:
        cat1: str
        cat2: str
        graph: Dict
    Returns:
        graph: Dict
    """
    cat1 = cat1.split(" + ")
    cat2 = cat2.split(" + ")
    
    if len(cat1) <= len(cat2):
        if (cat1 in cat2) & (cat1 != cat2):
            addEdge(graph, cat1, cat2)
    return graph
    

def graph2dict(graph, cats):
    """
    Args:
        graph: nx.Graph
        cats: np.array
    Returns:
        mydict: Dict
    """
    mydict = dict()
    for pbcat in pbcats:
        ncc = list(nx.node_connected_component(G, pbcat)) # Set[str] -> List[str]
        cat = " + ".join(ncc) # List[str] -> str
        cat = list(set(cat.split(" + "))) # str -> List[str]
        cat.sort()
        cat = " + ".join(cat) # List[str] -> str
        mydict[pbcat] = cat
    
    return mydict

In [34]:
# 
pbcats = tmp["pub_category"].unique()

In [38]:
pbcats = pbcats[:3]

In [70]:
pbcats

array(['survey of doctorate recipients + survey of graduate students and postdoctorates in science and engineering + survey of earned doctorates',
       'survey of earned doctorates', 'survey of doctorate recipients'],
      dtype=object)

In [71]:
graph = defaultdict(list)
for cat1 in pbcats:
    cat1 = cat1.split(" + ")
    for cat2 in pbcats:
        cat2 = cat2.split(" + ")
        
        if (len(cat1) <= len(cat2)) & (cat1 != cat2):
            for c1 in cat1:
                if c1 in cat2:
                    for c2 in cat2:
                        if c1 != c2:
                            addEdge(graph, c1, c2)
#outs = Parallel(n_jobs=-1)(delayed(cosim)(doc1, doc2) for doc2 in tqdm(labels) for doc1 in tqdm(labels))

In [72]:
graph

defaultdict(list,
            {'survey of doctorate recipients': ['survey of graduate students and postdoctorates in science and engineering',
              'survey of earned doctorates'],
             'survey of earned doctorates': ['survey of doctorate recipients',
              'survey of graduate students and postdoctorates in science and engineering']})

In [73]:
edges = generate_edges(graph)
G = nx.Graph()
G.add_edges_from(edges)

mydict = graph2dict(G, pbcats)

KeyError: ignored

In [ ]:
assert len(tmp) == num_id

In [ ]:
tmp["cleaned_label"].nunique()

262

In [ ]:
tmp["pub_category"].nunique()

159

In [ ]:
def get_cv(dataset, num_splits=None, col_target=None, col_group=None):
    """
    Args:
        dataset: pd.DataFrame
        num_splits: int
        col_target: str
        col_group: str
    Returns:
        folds: pd.DataFrame
    """
    X = dataset.index.values
    y = dataset[col_target].values
    groups = dataset[col_group].values

    group_kfold = GroupKFold(n_splits=num_splits)
    group_kfold.get_n_splits(X, y, groups)
    
    folds = pd.DataFrame()
    for i, (_, test_index) in enumerate(group_kfold.split(X, y, groups)):
        X_test = X[test_index]
        X_test = dataset[dataset.index.isin(X_test)]
        
        # Concat all and save at once
        X_test["fold"] = i+1
        folds = pd.concat([folds, X_test], ignore_index=True)
    
    return folds

In [ ]:
folds = get_cv(tmp, num_splits=5, col_target="cleaned_label", col_group="pub_category")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
col = "pub_category"
for i in range(5):
    train = folds[folds["fold"] != i+1]
    dev = folds[folds["fold"] == i+1]
    print(f"CV: {i+1} -------------------------------------------------------------------")
    print()
    print(f"#### train {col}:\n\n", np.sort(train[col].unique()))
    print()
    print(f"#### dev {col}:\n\n", np.sort(dev[col].unique()))
    print()
    print()
    assert train[col].nunique() + dev[col].nunique() == folds[col].nunique()

CV: 1 -------------------------------------------------------------------

#### train pub_category:

 ['aging integrated database' 'agricultural resources management survey'
 'agricultural resources management survey + arms farm financial and crop production practices'
 'agricultural resources management survey + census of agriculture'
 'agricultural resources management survey + census of agriculture + rural urban continuum codes'
 'agricultural resources management survey + program for the international assessment of adult competencies + higher education research and development survey + trends in international mathematics and science study'
 'agricultural resources management survey + rural urban continuum codes'
 'agricultural resources management survey + survey of industrial research and development'
 'alzheimers disease neuroimaging initiative + baltimore longitudinal study of aging'
 'arms farm financial and crop production practices'
 'baccalaureate and beyond longitudinal stu

In [ ]:
col = "cleaned_label"
for i in range(5):
    train = folds[folds["fold"] != i+1]
    dev = folds[folds["fold"] == i+1]
    print(f"CV: {i+1} -------------------------------------------------------------------")
    print()
    print(f"#### train {col}:\n\n", np.sort(train[col].unique()))
    print()
    print(f"#### dev {col}:\n\n", np.sort(dev[col].unique()))
    print()
    print()
    assert train[col].nunique() + dev[col].nunique() == folds[col].nunique()

CV: 1 -------------------------------------------------------------------

#### train cleaned_label:

 ['2019 ncov complete genome sequences' '2019 ncov genome sequence'
 '2019 ncov genome sequences'
 'adni|baltimore longitudinal study of aging blsa '
 'advanced national seismic system anss comprehensive catalog comcat '
 'advanced national seismic system comprehensive catalog'
 'aging integrated database' 'aging integrated database agid '
 'agricultural resource management survey'
 'agricultural resource management survey|rural urban continuum codes'
 'agricultural resources management survey'
 'agricultural resources management survey|arms farm financial and crop production practices'
 'agricultural resources management survey|census of agriculture'
 'anss comprehensive catalog' 'anss comprehensive earthquake catalog'
 'arms farm financial and crop production practices'
 'arms farm financial and crop production practices|agricultural resource management survey'
 'baccalaureate and be

***
# Get text & Export

In [ ]:
def get_text(filename, train=False):
    if train:
        df = pd.read_json(f'../input/coleridgeinitiative-show-us-the-data/train/{filename}.json')
    else:
        df = pd.read_json(f'../input/coleridgeinitiative-show-us-the-data/test/{filename}.json')

    text = " ".join(list(df['text']))
    return text

if 'google.colab' in sys.modules:
    folds.to_pickle(f"./{dname}/folds_pubcat.pkl")
    print("get_text should be run on Kaggle Platform")
    print("Please consider to use kagglenb006-get-text with folds_pubcat.pkl as its input...")
    print("Starting to upload folds_pubcat.pkl to Kaggle Platform...")
    if not os.path.exists(dname):
        !kaggle datasets init -p {dname}
        with open(f"{dname}/dataset-metadata.json", "r") as jsonFile:
            data = json.load(jsonFile)
        data["id"] = f"riow1983/{dname}"
        data["title"] = dname
        with open(f"{dname}/dataset-metadata.json", "w") as jsonFile:
            json.dump(data, jsonFile)
        !kaggle datasets create -p {dname}
    else:
        !kaggle datasets version -p {dname} -m "[Update] full version based on manual & cosine"

elif 'kaggle_web_client' in sys.modules:
    folds['text'] = folds['Id'].apply(lambda x: get_text(x, train=True))
    folds.to_pickle("./folds_pubcat.pkl")

get_text should be run on Kaggle Platform
Please consider to use kagglenb006-get-text with folds_pubcat.pkl as its input...
Starting to upload folds_pubcat.pkl to Kaggle Platform...
Starting upload for file outs.pkl
100% 132k/132k [00:03<00:00, 37.8kB/s]
Upload successful: outs.pkl (132KB)
Starting upload for file folds_pubcat.pkl
100% 2.45M/2.45M [00:05<00:00, 448kB/s]
Upload successful: folds_pubcat.pkl (2MB)
Dataset version is being created. Please check progress at /api/v1/datasets/status/riow1983/nb009-cv
